In [1]:
from pydantic_ai import Agent, RunContext
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()

load_dotenv()

True

In [76]:
from pydantic import BaseModel
from typing import Literal

def create_agent_output_class(list_of_next_agents: list[str]):
    class MASAgentResponse(BaseModel):
        goto: Literal[*list_of_next_agents]  # Unpacks list into Literal values
        response: str

        def __repr__(self):
            return (
                f"MASAgentResponse(response={self.response}, next_agent={self.goto})\n"
                f"  [next_agent must be one of: {list_of_next_agents}]"
            )

    return MASAgentResponse

so_next_agents = ['user', 'drill_creation_agent', 'curriculum_tracking_agent', 'grammar_explanation_agent']

supervisor_agent_system_prompt = """
You are the supervisor agent of a multi-agent system.

Your job is to:
- Delegate tasks to the appropriate agents
- Aggregate results
- Communicate with the user

You never perform tasks directly. You are only a coordinator.

If a request comes from the user, analyze it and break it down into subtasks. Delegate each task to the appropriate agent.
If a result comes from an agent, assess whether it fulfills the original user request or whether further delegation is needed.
If you communicate to an agent, instruct them as if you are instructing them directly.
If you communicate to the user, talk to them in a friendly, engaging, and educational manner.

Each agent in the system is specialized. Ensure agents do not exceed their scope. Delegate, route, and coordinate only.

---

You can communicate with the following agents in this system:
- `drill_creation_agent`: creates grammar, translation, and vocabulary drills when given enough information.
- `curriculum_tracking_agent`: tracks the user’s language learning progress and suggests the next learning objectives.
- `grammar_explanation_agent`: explains Spanish grammar concepts clearly and concisely.
- `user`: the user of the system.
"""

supervisor_agent = Agent(  
    'gpt-4o-mini',
    system_prompt=supervisor_agent_system_prompt,
    output_type=create_agent_output_class(so_next_agents)
)



drill_creation_next_agents = ['supervisor_agent', 'curriculum_tracking_agent', 'grammar_explanation_agent']

drill_creation_agent_system_prompt = """
You are an agent in a multi-agent system, responsible for **creating language learning drills** based on provided input.

🧠 Your domain includes:
- Creating grammar, translation, and vocabulary drills
- Formatting drills clearly and appropriately for the user's level

🚫 Your domain does *not* include:
- Tracking what the user has learned
- Deciding what the user should learn next
- Communicating directly with the user

🎯 If you lack the information needed to generate a drill (e.g., the user's proficiency level or learning goal), you must request that another agent provide this.

You are a tool in a larger system. Only perform actions within your domain. Delegate or request collaboration when needed.

---

You can communicate with the following agents in this system:

- `curriculum_tracking_agent`: tracks learning progress and can suggest what drill to create.
- `grammar_explanation_agent`: explains Spanish grammar concepts clearly and concisely.
- `supervisor_agent`: delegates tasks and oversees the system.

Only communicate with the agents listed here. Perform your task only when appropriate and fully informed.
"""

drill_creation_agent = Agent(
    'gpt-4o-mini',
    system_prompt=drill_creation_agent_system_prompt,
    output_type=create_agent_output_class(drill_creation_next_agents)
)



curriculum_tracking_next_agents = ['supervisor_agent', 'drill_creation_agent', 'grammar_explanation_agent']

curriculum_tracking_agent_system_prompt = """
You are an agent in a multi-agent system, responsible for **tracking the user's language learning progress** and **recommending what to learn next** based on that progress.

🧠 Your domain includes:
- Assessing what the user has or hasn’t learned
- Identifying knowledge gaps
- Recommending learning objectives

🚫 Your domain does *not* include:
- Creating learning materials (e.g. drills or exercises)
- Communicating directly with the user

🎯 If a learning objective should be acted on (e.g., a drill should be created), you must **recommend or instruct** that another agent take that action — you never take it yourself.

You communicate only through structured instructions to other agents in the system. Always stay within your scope, and delegate clearly when something is outside your role.

---

You can communicate with the following agents in this system:

- `drill_creation_agent`: creates drills for grammar, translation, or vocabulary, based on your recommendations.
- `grammar_explanation_agent`: explains Spanish grammar concepts clearly and concisely.
- `supervisor_agent`: coordinates the system and communicates with the user.

Only communicate with the agents listed here. Delegate clearly and appropriately.
"""

curriculum_tracking_agent = Agent(
    'gpt-4o-mini',
    system_prompt=curriculum_tracking_agent_system_prompt,
    output_type=create_agent_output_class(curriculum_tracking_next_agents)
)

@curriculum_tracking_agent.tool_plain
def get_grammar_concept_proficiency() -> str:
    return """
    Current user's level of Spanish proficiency in grammar:
    - Verb conjugation: not yet started
    - Past tense: not yet started
    - Present perfect: not yet started
    - Present participle: not yet started
    - Preterite: not yet started
    - Future: not yet started
    - Conditional: not yet started
    - Imperative: not yet started
    """



grammar_explanation_next_agents = ['supervisor_agent', 'drill_creation_agent', 'curriculum_tracking_agent']

grammar_explanation_agent_system_prompt = """
You are an agent in a multi-agent system. Your role is to **explain Spanish grammar concepts** clearly and concisely.

🧠 Your domain includes:
- Providing explanations for specific grammar concepts (e.g., verb conjugation, preterite tense, gender agreement)
- Giving relevant examples
- Using simple, clear, and beginner-friendly language

🚫 Your domain does *not* include:
- Creating drills or exercises
- Tracking user progress or recommending next steps
- Communicating directly with the user

🎯 You respond when asked to explain a grammar topic. You only act when requested by another agent — you do not make unsolicited suggestions or initiate tasks.

---

You can communicate with the following agents in this system:

- `supervisor_agent`: coordinates the overall workflow and communicates with the user
- `drill_creation_agent`: may ask for grammar explanations when generating a drill
- `curriculum_tracking_agent`: may ask for grammar concept explanations to support learning objectives

Only communicate with the agents listed here. Stay within your explanatory role and delegate anything outside your scope.
"""

grammar_explanation_agent = Agent(
    'gpt-4o-mini',
    system_prompt=grammar_explanation_agent_system_prompt,
    output_type=create_agent_output_class(grammar_explanation_next_agents)
)


input = 'I would like to practice my Spanish. I would like to learn about my next grammar concept.'
agent_mapping = {
    'supervisor_agent': supervisor_agent,
    'drill_creation_agent': drill_creation_agent,
    'curriculum_tracking_agent': curriculum_tracking_agent,
    'grammar_explanation_agent': grammar_explanation_agent
}

current_agent = 'supervisor_agent'

print(f"Starting {current_agent} with input: {input}")
print("--------------------------------\n\n")

history = []

def get_prompt(history, initial_input=input) -> str:
    if len(history) == 0:
        return f"The user has given the following request: {initial_input}."
    
    # Else
    new_prompt = f"The user has given the following initial request: {initial_input}.\n The following is the history of responses of the various agents that have acted so far on this request:\n"
    for agent, response, next_agent in history:
        new_prompt += f"Agent {agent} instructed agent {next_agent}: {response}\n"

    return new_prompt
        
for i in range(10):
    prompt = get_prompt(history)
    
    result = agent_mapping[current_agent].run_sync(prompt)

    print(f"Prompt to {current_agent}: {prompt}")
    print(f"{current_agent} returned:")
    print(f"Response: {result.output.response}\n")
    print(f"Next agent: {result.output.goto}")
    print("--------------------------------\n\n")

    if result.output.goto == 'user':
        break

    history.append((current_agent, result.output.response, result.output.goto))
    current_agent = result.output.goto

Starting supervisor_agent with input: I would like to practice my Spanish. I would like to learn about my next grammar concept.
--------------------------------


Prompt to supervisor_agent: The user has given the following request: I would like to practice my Spanish. I would like to learn about my next grammar concept..
supervisor_agent returned:
Response: Please provide the user with their current learning progress and suggest the next grammar concept they should learn.

Next agent: curriculum_tracking_agent
--------------------------------


Prompt to curriculum_tracking_agent: The user has given the following initial request: I would like to practice my Spanish. I would like to learn about my next grammar concept..
 The following is the history of responses of the various agents that have acted so far on this request:
Agent supervisor_agent instructed agent curriculum_tracking_agent: Please provide the user with their current learning progress and suggest the next grammar concept 

Agent(model=OpenAIModel(), name='drill_creation_agent', end_strategy='early', model_settings=None, output_type=<class '__main__.create_agent_output_class.<locals>.MASAgentResponse'>, instrument=None)